In [2]:
import pandas as pd
import numpy as np

### Analyst has prepared 100 different scenarios of price movements of an unknown product and saved them in file data_2.csv. Unit used is €/MWh.

In [3]:
df = pd.read_csv('../data/data_2.csv', parse_dates=["Datum"], index_col=['Datum'])
df.index = pd.to_datetime(df.index).normalize()
df.head()

,Scenarij_1,Scenarij_2,Scenarij_3,Scenarij_4,Scenarij_5,Scenarij_6,Scenarij_7,Scenarij_8,Scenarij_9,Scenarij_10,...,Scenarij_91,Scenarij_92,Scenarij_93,Scenarij_94,Scenarij_95,Scenarij_96,Scenarij_97,Scenarij_98,Scenarij_99,Scenarij_100
Datum,,,,,,,,,,,,,,,,,,,,,
2020-01-17,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,...,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00
2020-01-18,50.85,50.16,50.86,50.22,50.32,49.42,50.40,49.22,49.88,49.07,...,50.15,49.98,48.82,50.05,49.63,50.01,51.12,50.72,50.11,49.90
2020-01-19,51.94,50.58,50.91,50.14,50.57,48.72,50.03,48.65,50.62,49.28,...,50.92,50.50,49.62,50.18,49.30,49.44,51.44,50.49,51.50,49.53
2020-01-20,51.23,51.30,51.06,51.73,50.13,49.65,50.42,48.53,50.30,49.22,...,51.11,50.24,50.00,50.64,49.45,49.16,51.72,49.61,51.21,50.37
2020-01-21,52.38,50.60,51.63,50.95,50.50,50.62,51.35,48.42,50.14,48.84,...,51.00,49.59,50.65,51.11,49.86,48.72,51.31,49.85,51.17,49.34


#### a) What is the expected value of the product on 1.4.2020?

In [4]:
print(f"Expected value of the product on 1.4.2020 is {df[df.index == pd.to_datetime('2020-04-01')].mean().mean().item():.2f} €/MWh")

Expected value of the product on 1.4.2020 is 48.14 €/MWh


Colleagues are preparing an algorithm that is going to trade with this product. Number of trades is not limited, but algorithm can own at most one unit of the product at the time. Algorithm is not allowed to sell more units than it has bought. Due to an error made during the creation of the testing environment, the algorithm knows the price of the following day, so it always makes the best possible trades.

#### Calculate the

####  b) profit of the algorithm in scenario #77. Keep in mind that on the last trading day algorithm has to sell any purchased products that it has not already sold before. Profit is calculated as a difference between selling and buying price.

In [5]:
sc_77 = df.filter(like="77").copy()
sc_77

,Scenarij_77
Datum,
2020-01-17,50.00
2020-01-18,50.09
2020-01-19,50.80
2020-01-20,50.61
2020-01-21,50.79
...,...
2020-04-26,50.47
2020-04-27,49.21
2020-04-28,49.91


In [6]:
sc_77["Entry"] = np.where(sc_77["Scenarij_77"].shift(-1) > sc_77["Scenarij_77"], 1, 0)
sc_77["Exit"] = np.where(sc_77["Scenarij_77"].shift(-1) < sc_77["Scenarij_77"], -1, 0)

# kjer trenutna ali prejsma vr ni 1
sc_77["Entry"] = sc_77["Entry"].where((sc_77["Entry"].shift() != 1) | (sc_77["Entry"] != 1), 0)
sc_77["Exit"] = sc_77["Exit"].where((sc_77["Exit"].shift() != -1) | (sc_77["Exit"] !=-1), 0)
sc_77

,Scenarij_77,Entry,Exit
Datum,,,
2020-01-17,50.00,1,0
2020-01-18,50.09,0,0
2020-01-19,50.80,0,-1
2020-01-20,50.61,1,0
2020-01-21,50.79,0,-1
...,...,...,...
2020-04-26,50.47,0,0
2020-04-27,49.21,1,0
2020-04-28,49.91,0,0


In [7]:
sc_77["Trade_id"] = sc_77["Entry"].cumsum()
sc_77

,Scenarij_77,Entry,Exit,Trade_id
Datum,,,,
2020-01-17,50.00,1,0,1
2020-01-18,50.09,0,0,1
2020-01-19,50.80,0,-1,1
2020-01-20,50.61,1,0,2
2020-01-21,50.79,0,-1,2
...,...,...,...,...
2020-04-26,50.47,0,0,25
2020-04-27,49.21,1,0,26
2020-04-28,49.91,0,0,26


In [8]:
last_exit = sc_77.loc[sc_77["Exit"] == -1, "Trade_id"].max()
sc_77["Trade_id"] = sc_77["Trade_id"].where(sc_77["Trade_id"] <= last_exit)
sc_77.tail()

,Scenarij_77,Entry,Exit,Trade_id
Datum,,,,
2020-04-26,50.47,0,0,25.0
2020-04-27,49.21,1,0,NaN
2020-04-28,49.91,0,0,NaN
2020-04-29,50.52,0,0,NaN
2020-04-30,51.58,0,0,NaN


In [9]:
entry_prices = sc_77.loc[sc_77["Entry"] == 1].groupby("Trade_id")["Scenarij_77"].first()

exit_prices = sc_77.loc[sc_77["Exit"] == -1].groupby("Trade_id")["Scenarij_77"].first()
exit_prices

Trade_id
1.0     50.80
2.0     50.79
3.0     52.29
4.0     49.04
5.0     48.38
6.0     48.39
7.0     47.96
8.0     46.59
9.0     45.60
10.0    45.72
11.0    45.17
12.0    45.31
13.0    45.94
14.0    46.98
15.0    47.21
16.0    47.76
17.0    52.08
18.0    52.29
19.0    52.05
20.0    49.03
21.0    49.38
22.0    51.51
23.0    51.98
24.0    52.70
25.0    52.47
Name: Scenarij_77, dtype: float64

In [10]:
profit = (exit_prices - entry_prices).dropna()
profit_pct = ((exit_prices - entry_prices) / entry_prices).dropna()
profit.name = "Profit"
profit = profit.to_frame()
profit["Profits Comulative"] = profit["Profit"].cumsum()
profit["Profits pct"] = profit_pct
profit["Profits Comulative pct"] = profit["Profits pct"].cumsum()

profit

,Profit,Profits Comulative,Profits pct,Profits Comulative pct
Trade_id,,,,
1.0,0.80,0.80,0.016000,0.016000
2.0,0.18,0.98,0.003557,0.019557
3.0,2.73,3.71,0.055085,0.074641
4.0,0.27,3.98,0.005536,0.080178
5.0,0.23,4.21,0.004777,0.084954
6.0,0.37,4.58,0.007705,0.092659
7.0,0.79,5.37,0.016748,0.109407
8.0,0.97,6.34,0.021263,0.130670
9.0,0.28,6.62,0.006178,0.136848


In [12]:
print(f"Final absolute profit: {profit['Profits Comulative'].iloc[-1]:.2f} EUR")
print(f"Final relative profit: {profit['Profits Comulative pct'].iloc[-1]*100:.2f} %")

Final absolute profit: 24.78 EUR
Final relative profit: 51.88 %


#### Calculate the profit of the algorithm for other scenarios as well. Which scenario would be the most profitable for the c) algorithm?

In [13]:
def calculate_profits(df):

    results = []

    for col in df.columns:

        df_tmp = df[[col]].copy()

        df_tmp["Entry"] = np.where(df_tmp[col].shift(-1) > df_tmp[col], 1, 0)
        df_tmp["Exit"] = np.where(df_tmp[col].shift(-1) < df_tmp[col], -1, 0)

        # kjer trenutna ali prejsma vr ni 1
        df_tmp["Entry"] = df_tmp["Entry"].where((df_tmp["Entry"].shift() != 1) | (df_tmp["Entry"] != 1), 0)
        df_tmp["Exit"] = df_tmp["Exit"].where((df_tmp["Exit"].shift() != -1) | (df_tmp["Exit"] !=-1), 0)

        df_tmp["Trade_id"] = df_tmp["Entry"].cumsum()

        df_tmp["Trade_id"] = df_tmp["Trade_id"].where(df_tmp["Trade_id"] <= df_tmp.loc[df_tmp["Exit"] == -1, "Trade_id"].max())

        entry_prices = df_tmp.loc[df_tmp["Entry"] == 1].groupby("Trade_id")[col].first()

        exit_prices = df_tmp.loc[df_tmp["Exit"] == -1].groupby("Trade_id")[col].first()

        profit = (exit_prices - entry_prices).dropna()
        profit_pct = ((exit_prices - entry_prices) / entry_prices).dropna()

        total_profit = profit.sum()
        total_profit_pct = profit_pct.sum()

        results.append({
            "Scenario":col,
            "Total Profit":total_profit,
            "Total Profit_pct":total_profit_pct,
        })

    return pd.DataFrame(results).sort_values("Total Profit", ascending=False).reset_index(drop=True)


In [14]:
df_all = calculate_profits(df)
df_all

,Scenario,Total Profit,Total Profit_pct
0,Scenarij_48,39.81,0.710092
1,Scenarij_12,36.17,0.668732
2,Scenarij_44,35.29,0.637716
3,Scenarij_61,32.88,0.590924
4,Scenarij_20,30.67,0.630709
...,...,...,...
95,Scenarij_52,16.59,0.404572
96,Scenarij_11,15.16,0.341535
97,Scenarij_14,14.34,0.366732
98,Scenarij_38,14.06,0.351599


In [15]:
print(f"Most profitable Scenario is {df_all.iloc[0].Scenario} with total profit {df_all.iloc[0]['Total Profit']:.2f} EUR and relative profit {(df_all.iloc[0]['Total Profit_pct']*100):.2f} %")

Most profitable Scenario is Scenarij_48 with total profit 39.81 EUR and relative profit 71.01 %


#### Our trading partner offers us a call option, which would give us the right, but not the obligation, to buy the product on 1.4.2020 at 55 €/MWh, regardless of the market price on that day.

#### d) Using the given price movement scenarios, calculate the value of the offered option product.

In [199]:
option_date = pd.to_datetime("2020-04-01").date()
strike_price = 55.0

prices = df[df.index.date == option_date]

option_payoff = (prices - strike_price).clip(lower=0)

option_value = option_payoff.mean(axis=1).item()

print(f"Value of call option with strike price 55 € on 1.4.2020 is {option_value:.2f} €/MWh")

Value of call option with strike price 55 € on 1.4.2020 is 0.31 €/MWh
